In [1]:
#NNJM
#TODO
# Check to be sure slices are being done correctly
# Convolution instead of first dense hidden layer
# Put in character CNN
# Put in bi-dir LSTm
# Add in different loss function
# Add in argparse

In [2]:
#Import statements
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [17]:
#Parameters for the model
params = {}
params['minibatch'] = 50
params['emb-size'] = 200
params['hiddenstate-size'] = 512
params['source-vocab'] = 500
params['target-vocab'] = 600
params['datatype'] = tf.float32
params['init-method'] = 'uniform'
params['init-range'] = 0.01
params['source-window'] = 11
params['target-window'] = 4
params['use-char'] = False
params['seed'] = 1
params['dropout-rate'] = 0.0
params['loss'] = 'MLE' #('MLE','NCE','IS')
params['learning-rate'] = 0.5
params['val-check-rate'] = 0.5 #every fraction of the training set get val perplexity

#Character model params
char_params = {}
char_params['num-highway-layers'] = 4
char_params['char-emb-size'] = 25
char_params['filter-width'] = 7
char_params['longest-word'] = 30


params['char-params'] = char_params

In [18]:
assert params['val-check-rate']>0
assert params['val-check-rate']<=1
assert params['dropout-rate']>=0
assert params['dropout-rate']<=1
assert params['init-range']>0
assert params['minibatch']>0
assert params['learning-rate']>=0

In [4]:
#Interactive session
np.random.seed(seed=params['seed'])
sess = tf.InteractiveSession()

In [5]:
#Process the data, get the source-vocab and target-vocab variables


In [6]:
#Place holders for input and output data, first index is the minibatch size

#For the input the second dimension will be passed a
#     vector of size minibatch x (params['source-window']+params['target-window'])
with tf.device('/cpu:0'):
    input_indices = tf.placeholder(tf.int64, shape=[None, params['source-window']+params['target-window']])

#The output will be given a minibatch vector of correct indicies
with tf.device('/cpu:0'):
    correct_output = tf.placeholder(tf.int64,shape=[None])

#For passing in the learning rate and dropout rate
with tf.device('/cpu:0'):
    learning_rate = tf.placeholder(params['datatype'],shape=[])
    dropout_rate = tf.placeholder(params['datatype'],shape=[])

In [7]:
def param_init(params,shape=None,name=None,datatype=None):
    if datatype == None:
        datatype = params['datatype']
    assert shape != None,"Error shape cannot be None in param_init"
    if params['init-method'] == 'uniform':
        return tf.random_uniform(shape, minval=-1*params['init-range'],maxval=params['init-range'],\
                    dtype=datatype, seed=params['seed'], name=name)
    else:
        print "ERROR this init-method has not been created yet"

In [8]:
#Parameters for the source and target embeddings
with tf.device('/cpu:0'):
    source_emb_matrix = tf.Variable(param_init(params,shape=[params['source-vocab'],params['emb-size']],name='src-emb'))
    target_emb_matrix = tf.Variable(param_init(params,shape=[params['target-vocab'],params['emb-size']],name='tgt-emb'))

In [9]:
#Do the one-hot emebedding lookups
src_embed = tf.nn.embedding_lookup(source_emb_matrix, tf.slice(input_indices,[0,0],[-1,params['source-window']]))
tgt_embed = tf.nn.embedding_lookup(target_emb_matrix, tf.slice(input_indices,[0,params['source-window']],\
                                                                [-1,params['target-window']]))

In [10]:
#Now reshape to be able to feed through non-linearity
concat_embed = tf.concat(1, [src_embed, tgt_embed])
concat_embed = tf.reshape(concat_embed,[-1,params['emb-size']*(params['source-window']+params['target-window'])])
concat_embed = tf.nn.dropout(concat_embed, 1-params['dropout-rate']) #Pass in the keep prob

In [11]:
#First Layer
with tf.device('/cpu:0'):
    layer_1_weights = tf.Variable(param_init(params,\
        shape=[params['emb-size']*(params['source-window']+params['target-window']),params['hiddenstate-size']],name='lyr-1'))
    layer_1_bias = tf.Variable(param_init(params,shape=[params['hiddenstate-size']],name='lyr-1-bias'))
layer_1_output = tf.nn.relu(tf.matmul(concat_embed,layer_1_weights)+layer_1_bias)
layer_1_output = tf.nn.dropout(layer_1_output, 1-params['dropout-rate']) #Pass in the keep prob

In [12]:
#Second layer
with tf.device('/cpu:0'):
    layer_2_weights = tf.Variable(param_init(params,shape=[params['hiddenstate-size'],params['hiddenstate-size']],\
                            name='lyr-2'))
    layer_2_bias = tf.Variable(param_init(params,shape=[params['hiddenstate-size']],name='lyr-2-bias'))
layer_2_output = tf.nn.relu(tf.matmul(layer_1_output,layer_2_weights)+layer_2_bias)
layer_2_output = tf.nn.dropout(layer_2_output, 1-params['dropout-rate']) #Pass in the keep prob

In [13]:
#Softmax layer
with tf.device('/cpu:0'):
    softmax_weights = tf.Variable(param_init(params,shape=[params['hiddenstate-size'],params['target-vocab']],\
                                  name='softmax-weights')) 
    softmax_bias = tf.Variable(param_init(params,shape=[params['target-vocab']],name='softmax-bias'))
unscaled_final_output = tf.matmul(layer_2_output, softmax_weights) + softmax_bias
loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(unscaled_final_output,correct_output))

In [14]:
#Optimizer
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [15]:
#Additional variables for tracking loss


